# Environment Setup and Imports

In [ ]:
import os
import sys
import time
from datetime import datetime
import platform
import warnings
warnings.filterwarnings('ignore')

# Platform detection and configuration
PLATFORM = platform.system()
IS_EMR = 'EMR' in os.environ.get('AWS_EXECUTION_ENV', '')
print(f"Running on: {PLATFORM}")
print(f"Is EMR: {IS_EMR}")
print(f"Python version: {sys.version}")

# Set environment variables based on platform
if PLATFORM == "Windows":
    os.environ["PYSPARK_PYTHON"] = sys.executable
    os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
    if "JAVA_HOME" not in os.environ:
        java_paths = ["C:\\Program Files\\Java\\jdk-11", "C:\\Zulu\\zulu-11"]
        for path in java_paths:
            if os.path.exists(path):
                os.environ["JAVA_HOME"] = path
                break
else:
    os.environ["PYSPARK_PYTHON"] = sys.executable
    os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
    if "JAVA_HOME" not in os.environ:
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Import required libraries
import numpy as np
import pandas as pd
from PIL import Image
import io
from typing import List, Tuple, Dict
import json
import pickle

# PySpark imports
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, udf, element_at, split, count, when, lit
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType, BooleanType, BinaryType
from pyspark.ml.feature import StandardScaler, VectorAssembler, StringIndexer, PCA
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT

# TensorFlow imports
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


# Configuration Parameters

In [ ]:
# Configuration parameters
CONFIG = {
    "batch_size": 64,  # Increased batch size
    "image_size": (224, 224),
    "num_features": 1280,
    "test_split": 0.2,
    "seed": 42,
    # Random Forest parameters - optimized
    "num_trees": 200,  # Increased from 100
    "max_depth": 20,   # Increased from 15
    "min_instances_per_node": 1,  # Reduced from 2
    "feature_subset_strategy": "sqrt",
    # PCA parameters - optimized
    "pca_components": 100,  # Increased from 50
    "pca_variance_threshold": 0.99  # Increased from 0.95
}



# Path configuration

DATA_PATH = "s3://oc-p8-charbonneau-fruits-input/Test1"
RESULTS_PATH = "s3://oc-p8-charbonneau-fruits-output/Results"
MODEL_PATH = "s3://oc-p8-charbonneau-fruits-input/models"
os.makedirs(MODEL_PATH, exist_ok=True)

print(f"Data path: {DATA_PATH}")
print(f"Results path: {RESULTS_PATH}")
print(f"Model path: {MODEL_PATH}")


# Disable GPU

In [ ]:
# Disable GPU for TensorFlow to avoid memory issues in distributed processing
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Force TensorFlow to use CPU
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')


# Initialize Spark Session

In [ ]:
# Start timing
start_time = time.time()
initialization_start = time.time()

# Create Spark session with optimized configuration
def create_spark_session():
    builder = SparkSession.builder \
        .appName("FruitClassification_TransferLearning_CrossPlatform") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "10000") \
        .config("spark.python.worker.memory", "2g") \
        .config("spark.python.worker.reuse", "true")
    
    if not IS_EMR:
        builder = builder.master("local[*]") \
            .config("spark.driver.memory", "8g") \
            .config("spark.executor.memory", "8g") \
            .config("spark.driver.maxResultSize", "4g") \
            .config("spark.sql.shuffle.partitions", "50") \
            .config("spark.default.parallelism", "50")
    else:
        builder = builder \
            .config("spark.sql.shuffle.partitions", "200") \
            .config("spark.default.parallelism", "200") \
            .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
            .config("spark.kryoserializer.buffer.max", "1024m")
    
    return builder.getOrCreate()

spark = create_spark_session()
sc = spark.sparkContext
print(f"Spark version: {spark.version}")
print(f"Number of executors: {sc._jsc.sc().getExecutorMemoryStatus().size()}")
print(f"Default parallelism: {sc.defaultParallelism}")
initialization_time = time.time() - initialization_start
print(f"\nSpark initialization time: {initialization_time:.2f} seconds")


# Load Pre-trained Model and Broadcast

In [ ]:
# Load and Broadcast TensorFlow Model Weights
print("="*80)
print("TENSORFLOW MODEL BROADCASTING DEMONSTRATION")
print("="*80)

broadcast_start = time.time()

# Load MobileNetV2 model
print("\n1. Loading MobileNetV2 pre-trained model...")
model_creation_start = time.time()

# Create base model without top layers
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Freeze the base model
base_model.trainable = False
model_creation_time = time.time() - model_creation_start
print(f"   Model creation time: {model_creation_time:.2f} seconds")

# Extract model weights and architecture
print("\n2. Extracting model weights and architecture...")
extraction_start = time.time()

model_weights = base_model.get_weights()
model_config = base_model.get_config()

# Calculate model size
weights_size = sum([w.nbytes for w in model_weights])
print(f"   Number of weight arrays: {len(model_weights)}")
print(f"   Total weights size: {weights_size / (1024*1024):.2f} MB")

extraction_time = time.time() - extraction_start
print(f"   Extraction time: {extraction_time:.2f} seconds")

# Serialize weights for broadcasting
print("\n3. Serializing weights for broadcast...")
serialization_start = time.time()

serialized_weights = pickle.dumps(model_weights)
serialized_size = len(serialized_weights)

serialization_time = time.time() - serialization_start
print(f"   Serialized size: {serialized_size / (1024*1024):.2f} MB")
print(f"   Serialization time: {serialization_time:.2f} seconds")

# Broadcast to all nodes
print("\n4. Broadcasting weights to all Spark nodes...")
broadcast_operation_start = time.time()

# Clear any existing broadcasts
if 'broadcast_weights' in locals():
    try:
        broadcast_weights.unpersist()
    except:
        pass
if 'broadcast_config' in locals():
    try:
        broadcast_config.unpersist()
    except:
        pass

# Broadcast the serialized weights
broadcast_weights = sc.broadcast(serialized_weights)
broadcast_config = sc.broadcast(model_config)

broadcast_operation_time = time.time() - broadcast_operation_start
print(f"   Broadcast operation time: {broadcast_operation_time:.2f} seconds")

# Verify broadcast on executors
print("\n5. Verifying broadcast on executors...")
verification_start = time.time()

def verify_broadcast_on_executor(partition_id):
    """Verify that broadcast variables are accessible on executor"""
    import tensorflow as tf
    import pickle
    import os
    
    # Force CPU usage
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
    try:
        # Access broadcast variables
        weights = pickle.loads(broadcast_weights.value)
        config = broadcast_config.value
        
        # Recreate model to verify
        with tf.device('/CPU:0'):
            test_model = tf.keras.Model.from_config(config)
            test_model.set_weights(weights)
        
        return {
            'success': True,
            'partition_id': partition_id,
            'weights_count': len(weights),
            'config_layers': len(config.get('layers', [])),
            'hostname': os.uname()[1] if hasattr(os, 'uname') else 'unknown'
        }
    except Exception as e:
        return {
            'success': False,
            'partition_id': partition_id,
            'error': str(e),
            'hostname': os.uname()[1] if hasattr(os, 'uname') else 'unknown'
        }

# Test on multiple partitions to verify distribution
num_partitions = min(sc.defaultParallelism, 4)
verification_rdd = sc.parallelize(range(num_partitions), num_partitions)
verification_results = verification_rdd.mapPartitionsWithIndex(
    lambda idx, iterator: [verify_broadcast_on_executor(idx)]
).collect()

verification_time = time.time() - verification_start
print(f"   Verification time: {verification_time:.2f} seconds")

# Display verification results
print("\n6. Broadcast Verification Results:")
success_count = 0
for result in verification_results:
    if result['success']:
        success_count += 1
        print(f"   Partition {result['partition_id']}: ✓ Success - "
              f"{result['weights_count']} weight arrays, "
              f"{result['config_layers']} layers on {result['hostname']}")
    else:
        print(f"   Partition {result['partition_id']}: ✗ Failed - {result['error']}")

# Calculate total broadcast time
total_broadcast_time = time.time() - broadcast_start

# Summary
print("\n" + "="*80)
print("BROADCAST SUMMARY")
print("="*80)
print(f"Model type: MobileNetV2 (Transfer Learning)")
print(f"Original model size: {weights_size / (1024*1024):.2f} MB")
print(f"Broadcast size: {serialized_size / (1024*1024):.2f} MB")
print(f"Compression ratio: {weights_size / serialized_size:.2f}x")
print(f"Number of executors: {sc._jsc.sc().getExecutorMemoryStatus().size()}")
print(f"Broadcast verified on {success_count}/{len(verification_results)} partitions")
print(f"\nTiming breakdown:")
print(f"  - Model creation: {model_creation_time:.2f}s")
print(f"  - Weight extraction: {extraction_time:.2f}s")
print(f"  - Serialization: {serialization_time:.2f}s")
print(f"  - Broadcast operation: {broadcast_operation_time:.2f}s")
print(f"  - Verification: {verification_time:.2f}s")
print(f"  - Total time: {total_broadcast_time:.2f}s")
print("="*80)

# Store timing for final summary
model_load_time = total_broadcast_time

# Additional demonstration: Show that broadcast is accessible in transformations
print("\n7. Testing broadcast in a transformation...")
test_start = time.time()

def test_broadcast_in_transformation(x):
    """Test accessing broadcast in a transformation"""
    try:
        weights = pickle.loads(broadcast_weights.value)
        config = broadcast_config.value
        return f"Success: {len(weights)} weights, {len(config.get('layers', []))} layers"
    except Exception as e:
        return f"Error: {str(e)}"

# Test on a small RDD
test_rdd = sc.parallelize([1, 2, 3], 3)
test_results = test_rdd.map(test_broadcast_in_transformation).collect()

test_time = time.time() - test_start
print(f"   Transformation test time: {test_time:.2f} seconds")
print("   Results from each partition:")
for i, result in enumerate(test_results):
    print(f"     Partition {i}: {result}")

print("\n" + "="*80)
print("Broadcast successfully distributed and verified!")
print("="*80)


# Load and Prepare Image Data

In [ ]:
# Load images
loading_start = time.time()
print(f"Loading images from: {DATA_PATH}")

# Read images with labels from folder structure
image_read_start = time.time()
images_df = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.jpg") \
    .option("recursiveFileLookup", "true") \
    .load(DATA_PATH)
image_read_time = time.time() - image_read_start
print(f"Image reading time: {image_read_time:.2f} seconds")

# Extract labels from folder names
label_extraction_start = time.time()
images_df = images_df.withColumn('label', element_at(split(col('path'), '/'), -2))
label_extraction_time = time.time() - label_extraction_start
print(f"Label extraction time: {label_extraction_time:.2f} seconds")

# Add image validation
validation_start = time.time()
def validate_image(content):
    try:
        img = Image.open(io.BytesIO(content))
        if img.size[0] > 0 and img.size[1] > 0:
            return True
        return False
    except:
        return False

validate_image_udf = udf(validate_image, BooleanType())
images_df = images_df.withColumn('is_valid', validate_image_udf(col('content')))

# Filter out corrupted images
images_df = images_df.filter(col('is_valid') == True).drop('is_valid')
validation_time = time.time() - validation_start
print(f"Image validation time: {validation_time:.2f} seconds")

# Cache the dataframe
cache_start = time.time()
images_df.cache()
cache_time = time.time() - cache_start
print(f"Caching time: {cache_time:.2f} seconds")

# Count images per class
count_start = time.time()
image_count = images_df.count()
class_distribution = images_df.groupBy('label').count().orderBy('label').collect()
count_time = time.time() - count_start
print(f"Counting time: {count_time:.2f} seconds")

print(f"\nTotal valid images loaded: {image_count}")
print("\nClass distribution:")
for row in class_distribution:
    print(f"  {row['label']}: {row['count']} images")

loading_time = time.time() - loading_start
print(f"\nTotal data loading time: {loading_time:.2f} seconds")


# Data Augmentation

In [ ]:
# Data Augmentation for training data
def augment_image(image_bytes):
    """Apply data augmentation to improve model generalization"""
    try:
        img = Image.open(io.BytesIO(image_bytes))
        img = img.convert('RGB')
        
        # Random rotation
        if np.random.random() > 0.5:
            angle = np.random.uniform(-20, 20)
            img = img.rotate(angle, fillcolor=(255, 255, 255))
        
        # Random horizontal flip
        if np.random.random() > 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
        
        # Random brightness adjustment
        if np.random.random() > 0.5:
            from PIL import ImageEnhance
            enhancer = ImageEnhance.Brightness(img)
            img = enhancer.enhance(np.random.uniform(0.8, 1.2))
        
        # Convert back to bytes
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG')
        return img_byte_arr.getvalue()
    except:
        return image_bytes

# Apply augmentation to training data only (do this after train/test split)
augment_udf = udf(augment_image, BinaryType())


# Feature Extraction with Transfer Learning

In [ ]:
# Feature Extraction with Transfer Learning
from pyspark import StorageLevel  # Add this import

# Start timing for feature extraction
feature_extraction_start = time.time()

def extract_mobilenet_features_batch(image_bytes_list):
    """Extract features for a batch of images using MobileNetV2"""
    try:
        # Recreate model from broadcast weights - with CPU only
        with tf.device('/CPU:0'):
            weights = pickle.loads(broadcast_weights.value)
            config = broadcast_config.value
            
            # Recreate the model
            base_model = tf.keras.Model.from_config(config)
            base_model.set_weights(weights)
            
            # Process batch
            batch_features = []
            for image_bytes in image_bytes_list:
                try:
                    # Load and preprocess image
                    img = Image.open(io.BytesIO(image_bytes))
                    img = img.convert('RGB')
                    img = img.resize((224, 224))
                    
                    # Convert to array and preprocess
                    img_array = image.img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = preprocess_input(img_array)
                    
                    # Extract features
                    features = base_model.predict(img_array, verbose=0)
                    batch_features.append(features[0].tolist())
                except Exception as e:
                    # Return zeros if error for this image
                    batch_features.append([0.0] * CONFIG["num_features"])
            
            return batch_features
        
    except Exception as e:
        # Return zeros for entire batch if error
        return [[0.0] * CONFIG["num_features"]] * len(image_bytes_list)

# Process images in smaller batches to avoid memory issues
def process_partition(iterator):
    """Process a partition of images"""
    # Convert iterator to list
    rows = list(iterator)
    if not rows:
        return iter([])
    
    # Process in smaller batches
    batch_size = 32  # Reduced batch size
    result_rows = []
    
    for i in range(0, len(rows), batch_size):
        batch_rows = rows[i:i+batch_size]
        image_bytes_list = [row.content for row in batch_rows]
        
        # Extract features for the batch
        features_list = extract_mobilenet_features_batch(image_bytes_list)
        
        # Create result rows
        for j, row in enumerate(batch_rows):
            result_rows.append(Row(
                path=row.path,
                label=row.label,
                features=features_list[j]
            ))
    
    return iter(result_rows)

# Apply batch processing with reduced partitions
optimal_partitions = min(sc.defaultParallelism, 20)  # Reduced from 50
images_df_repartitioned = images_df.repartition(optimal_partitions)
features_rdd = images_df_repartitioned.rdd.mapPartitions(process_partition)
features_df = spark.createDataFrame(features_rdd)

# Convert to vector format
array_to_vector_udf = udf(lambda x: Vectors.dense(x) if x else Vectors.dense([0.0] * CONFIG["num_features"]), VectorUDT())
features_vector_df = features_df.select(
    col("path"),
    col("label"),
    array_to_vector_udf(col("features")).alias("features_vector")
)

# Cache with disk spillover to avoid memory issues
features_vector_df.persist(StorageLevel.MEMORY_AND_DISK)

# Calculate feature extraction time correctly
feature_extraction_time = time.time() - feature_extraction_start
print(f"\nFeature extraction time: {feature_extraction_time:.2f} seconds")

# Force computation and show count
processed_count = features_vector_df.count()
print(f"Successfully extracted features for {processed_count} images")


# PCA Analysis

In [ ]:
# PCA Analysis
pca_start = time.time()
print("Performing PCA analysis...")

# Prepare data with label indexing
label_indexer = StringIndexer(inputCol="label", outputCol="label_index")
label_indexer_model = label_indexer.fit(features_vector_df)
indexed_df = label_indexer_model.transform(features_vector_df)

# Standardize features
scaler = StandardScaler(
    inputCol="features_vector",
    outputCol="scaled_features",
    withStd=True,
    withMean=False
)
scaler_model = scaler.fit(indexed_df)
scaled_df = scaler_model.transform(indexed_df)

# Apply PCA
pca = PCA(k=CONFIG["pca_components"], inputCol="scaled_features", outputCol="pca_features")
pca_model = pca.fit(scaled_df)
pca_df = pca_model.transform(scaled_df)

# Calculate explained variance
explained_variance = pca_model.explainedVariance.toArray()
cumulative_variance = np.cumsum(explained_variance)

print(f"\nPCA Results:")
print(f"Components: {CONFIG['pca_components']}")
print(f"Total variance explained: {cumulative_variance[-1]:.2%}")

# Find number of components for threshold
n_components_threshold = np.argmax(cumulative_variance >= CONFIG["pca_variance_threshold"]) + 1
print(f"Components needed for {CONFIG['pca_variance_threshold']:.0%} variance: {n_components_threshold}")

pca_time = time.time() - pca_start
print(f"\nPCA analysis time: {pca_time:.2f} seconds")


# Visualize PCA Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create 2D PCA for visualization
pca_2d = PCA(k=2, inputCol="scaled_features", outputCol="pca_2d")
pca_2d_model = pca_2d.fit(scaled_df)
pca_2d_result = pca_2d_model.transform(scaled_df)

# Convert to Pandas for visualization
pca_viz_df = pca_2d_result.select("label", "pca_2d").toPandas()
pca_viz_df['pca_1'] = pca_viz_df['pca_2d'].apply(lambda x: float(x[0]))
pca_viz_df['pca_2'] = pca_viz_df['pca_2d'].apply(lambda x: float(x[1]))

# Plot PCA visualization
plt.figure(figsize=(12, 8))
sns.scatterplot(data=pca_viz_df, x='pca_1', y='pca_2', hue='label', 
                palette='viridis', alpha=0.6, s=50)
plt.title('PCA Visualization of MobileNetV2 Features', fontsize=16)
plt.xlabel(f'First Principal Component')
plt.ylabel(f'Second Principal Component')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Plot explained variance
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance) + 1), explained_variance)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Component')
plt.xticks(range(1, len(explained_variance) + 1, 5))

plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'bo-')
plt.axhline(y=CONFIG["pca_variance_threshold"], color='r', linestyle='--', 
            label=f'{CONFIG["pca_variance_threshold"]:.0%} threshold')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.legend()
plt.xticks(range(0, len(cumulative_variance) + 1, 5))

plt.tight_layout()
plt.show()


# Check similar fruits

In [ ]:
# Find and visualize similar fruits in PCA space
from pyspark.sql.functions import sqrt, pow as spark_pow
from pyspark.ml.linalg import Vectors
import matplotlib.pyplot as plt
from PIL import Image
import io

print("Finding similar fruits in PCA space...")

# First, let's get the PCA features with paths
# Use pca_df instead of pca_result
pca_with_paths = pca_df.select("path", "label", "pca_features", "label_index")

# Convert PCA features to individual columns for distance calculation
def extract_pca_components(vector):
    return float(vector[0]), float(vector[1])

extract_pca_udf = udf(extract_pca_components, ArrayType(FloatType()))

# Add PCA components as separate columns
pca_with_components = pca_with_paths.select(
    col("path"),
    col("label"),
    col("label_index"),
    col("pca_features"),
    element_at(extract_pca_udf(col("pca_features")), 1).alias("pca_1"),
    element_at(extract_pca_udf(col("pca_features")), 2).alias("pca_2")
)

# Cache for performance
pca_with_components.cache()

# Function to find nearest neighbors in PCA space
def find_similar_fruits(df, num_examples=5, num_neighbors=5):
    """Find examples of different fruits that are close in PCA space"""
    
    # Get a sample point from each class
    sample_points = []
    labels_list = df.select("label").distinct().collect()
    
    for label_row in labels_list[:num_examples]:  # Limit to num_examples classes
        label = label_row["label"]
        # Get one sample from this class
        sample = df.filter(col("label") == label).limit(1).collect()[0]
        if sample:
            sample_points.append({
                "label": label,
                "path": sample["path"],
                "pca_1": sample["pca_1"],
                "pca_2": sample["pca_2"]
            })
    
    # For each sample point, find nearest neighbors from different classes
    similar_pairs = []
    
    for sample in sample_points:
        # Calculate distances to all other points
        distances_df = df.filter(col("label") != sample["label"]).select(
            col("path"),
            col("label"),
            col("pca_1"),
            col("pca_2"),
            sqrt(
                spark_pow(col("pca_1") - lit(sample["pca_1"]), 2) + 
                spark_pow(col("pca_2") - lit(sample["pca_2"]), 2)
            ).alias("distance")
        )
        
        # Get top N nearest neighbors
        nearest = distances_df.orderBy("distance").limit(num_neighbors).collect()
        
        similar_pairs.append({
            "source": sample,
            "neighbors": nearest
        })
    
    return similar_pairs

# Find similar fruits
similar_fruits = find_similar_fruits(pca_with_components, num_examples=5, num_neighbors=3)

# Visualize the similar fruits
fig, axes = plt.subplots(len(similar_fruits), 4, figsize=(16, 4*len(similar_fruits)))
if len(similar_fruits) == 1:
    axes = axes.reshape(1, -1)

for idx, pair in enumerate(similar_fruits):
    source = pair["source"]
    neighbors = pair["neighbors"]
    
    # Load and display source image
    try:
        # Read the source image
        source_img_df = images_df.filter(col("path") == source["path"]).select("content").collect()
        if source_img_df:
            source_img = Image.open(io.BytesIO(source_img_df[0]["content"]))
            axes[idx, 0].imshow(source_img)
            axes[idx, 0].set_title(f"Source: {source['label']}\nPCA: ({source['pca_1']:.2f}, {source['pca_2']:.2f})", 
                                  fontsize=10)
            axes[idx, 0].axis('off')
    except:
        axes[idx, 0].text(0.5, 0.5, f"Source: {source['label']}", ha='center', va='center')
        axes[idx, 0].axis('off')
    
    # Display nearest neighbors
    for j, neighbor in enumerate(neighbors[:3]):  # Show top 3 neighbors
        try:
            # Read the neighbor image
            neighbor_img_df = images_df.filter(col("path") == neighbor["path"]).select("content").collect()
            if neighbor_img_df:
                neighbor_img = Image.open(io.BytesIO(neighbor_img_df[0]["content"]))
                axes[idx, j+1].imshow(neighbor_img)
                axes[idx, j+1].set_title(f"{neighbor['label']}\nDist: {neighbor['distance']:.3f}", 
                                        fontsize=10)
                axes[idx, j+1].axis('off')
        except:
            axes[idx, j+1].text(0.5, 0.5, f"{neighbor['label']}", ha='center', va='center')
            axes[idx, j+1].axis('off')

plt.suptitle('Similar Fruits in PCA Space\n(Source fruit on left, nearest neighbors from different classes on right)', 
             fontsize=14)
plt.tight_layout()
plt.show()

# Create a scatter plot highlighting the similar pairs
plt.figure(figsize=(12, 8))

# Plot all points
pca_viz_df = pca_with_components.select("label", "pca_1", "pca_2").toPandas()
sns.scatterplot(data=pca_viz_df, x='pca_1', y='pca_2', hue='label', 
                palette='viridis', alpha=0.3, s=30, legend=False)

# Highlight the similar pairs
colors = ['red', 'blue', 'green', 'orange', 'purple']
for idx, pair in enumerate(similar_fruits[:5]):
    source = pair["source"]
    neighbors = pair["neighbors"]
    
    # Plot source point
    plt.scatter(source['pca_1'], source['pca_2'], 
               color=colors[idx % len(colors)], s=200, marker='*', 
               edgecolors='black', linewidth=2,
               label=f"Source: {source['label']}")
    
    # Plot connections to neighbors
    for neighbor in neighbors[:3]:
        plt.plot([source['pca_1'], neighbor['pca_1']], 
                [source['pca_2'], neighbor['pca_2']], 
                color=colors[idx % len(colors)], alpha=0.5, linestyle='--')
        plt.scatter(neighbor['pca_1'], neighbor['pca_2'], 
                   color=colors[idx % len(colors)], s=100, marker='o',
                   edgecolors='black', linewidth=1)

plt.title('PCA Space: Similar Fruits from Different Classes', fontsize=16)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Print summary of findings
print("\nSummary of Similar Fruits in PCA Space:")
print("="*60)
for idx, pair in enumerate(similar_fruits):
    source = pair["source"]
    print(f"\nSource fruit: {source['label']}")
    print(f"PCA coordinates: ({source['pca_1']:.3f}, {source['pca_2']:.3f})")
    print("Nearest neighbors from different classes:")
    for j, neighbor in enumerate(pair["neighbors"][:3]):
        print(f"  {j+1}. {neighbor['label']} - Distance: {neighbor['distance']:.3f}")
print("="*60)


# Random Forest

In [ ]:
# Simple Random Forest Training without Cross-Validation
training_start = time.time()

# Prepare train/test split
train_df, test_df = pca_df.randomSplit([1-CONFIG["test_split"], CONFIG["test_split"]], 
                                       seed=CONFIG["seed"])

# Count samples
train_count = train_df.count()
test_count = test_df.count()
print(f"Training samples: {train_count}")
print(f"Test samples: {test_count}")

# Define Random Forest with optimized parameters for fruit classification
rf = RandomForestClassifier(
    featuresCol="pca_features",
    labelCol="label_index",
    numTrees=150,  # Good balance between performance and speed
    maxDepth=30,   # Prevents overfitting while maintaining accuracy
    minInstancesPerNode=2,  # Helps with generalization
    featureSubsetStrategy="sqrt",  # Standard for classification
    seed=CONFIG["seed"],
    maxBins=32,  # Default value, good for most cases
    subsamplingRate=0.8  # Helps prevent overfitting
)

# Train the model
print("\nTraining Random Forest model...")
model_fit_start = time.time()
rf_model = rf.fit(train_df)
model_fit_time = time.time() - model_fit_start

print(f"Model training completed in {model_fit_time:.2f} seconds")
training_time = time.time() - training_start
print(f"Total training time: {training_time:.2f} seconds")

# Print model summary
print(f"\nRandom Forest Model Summary:")
print(f"- Number of trees: {rf.getNumTrees()}")
print(f"- Max depth: {rf.getMaxDepth()}")
print(f"- Feature subset strategy: {rf.getFeatureSubsetStrategy()}")
print(f"- Number of features: {CONFIG['pca_components']}")
print(f"- Number of classes: {len(label_indexer_model.labels)}")


# Class balancing

In [ ]:
# Handle class imbalance
from pyspark.sql.functions import col, count, min as spark_min

# Calculate class weights
class_counts = train_df.groupBy("label_index").agg(count("*").alias("count"))
min_count = class_counts.agg(spark_min("count")).collect()[0][0]

# Create balanced dataset by oversampling minority classes
balanced_dfs = []
for row in class_counts.collect():
    label = row["label_index"]
    current_count = row["count"]
    oversample_ratio = min_count / current_count
    
    class_df = train_df.filter(col("label_index") == label)
    if oversample_ratio < 1:
        # Oversample minority class
        oversampled_df = class_df.sample(withReplacement=True, fraction=1.0/oversample_ratio, seed=CONFIG["seed"])
        balanced_dfs.append(oversampled_df)
    else:
        balanced_dfs.append(class_df)

# Combine balanced datasets
from functools import reduce
balanced_train_df = reduce(lambda df1, df2: df1.union(df2), balanced_dfs)


# Train Classification Model

In [ ]:
# Prepare data for training
preparation_start = time.time()

# Check if train_df already exists from cross-validation
if 'train_df' not in locals():
    split_start = time.time()
    # Use PCA features for training
    train_df, test_df = pca_df.randomSplit([1-CONFIG["test_split"], CONFIG["test_split"]], 
                                           seed=CONFIG["seed"])
    split_time = time.time() - split_start
    print(f"Train/test split time: {split_time:.2f} seconds")

count_start = time.time()
train_count = train_df.count()
test_count = test_df.count()
count_time = time.time() - count_start
print(f"Sample counting time: {count_time:.2f} seconds")

print(f"\nTraining samples: {train_count}")
print(f"Test samples: {test_count}")

# Train Random Forest on PCA features (skip if using cross-validation model)
if 'rf_model' not in locals():
    training_start = time.time()
    print("\nTraining Random Forest model on PCA features...")
    
    rf = RandomForestClassifier(
        featuresCol="pca_features",
        labelCol="label_index",
        numTrees=CONFIG["num_trees"],
        maxDepth=CONFIG["max_depth"],
        minInstancesPerNode=CONFIG["min_instances_per_node"],
        featureSubsetStrategy=CONFIG["feature_subset_strategy"],
        seed=CONFIG["seed"]
    )
    
    # Train model
    model_fit_start = time.time()
    rf_model = rf.fit(train_df)
    model_fit_time = time.time() - model_fit_start
    print(f"Model fitting time: {model_fit_time:.2f} seconds")
    
    training_time = time.time() - training_start
    print(f"Total model training time: {training_time:.2f} seconds")
else:
    print("\nUsing cross-validated model with best parameters")
    training_time = 0  # Already measured in cross-validation

preparation_time = time.time() - preparation_start
print(f"Total preparation time: {preparation_time:.2f} seconds")


# Feature Importance Analysis

In [ ]:
# Feature importance analysis
feature_importances = rf_model.featureImportances.toArray()
top_features_idx = np.argsort(feature_importances)[-20:][::-1]

print("\nTop 20 Most Important PCA Components:")
for idx in top_features_idx:
    print(f"PCA Component {idx}: {feature_importances[idx]:.4f}")


# Evaluate Model

In [ ]:
# Evaluate model
evaluation_start = time.time()
print("Evaluating model performance...")

# Make predictions
prediction_start = time.time()
predictions = rf_model.transform(test_df)
prediction_time = time.time() - prediction_start
print(f"Prediction time: {prediction_time:.2f} seconds")

# Calculate metrics
metrics_start = time.time()
evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="prediction", metricName="accuracy"
)
evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="prediction", metricName="f1"
)
evaluator_precision = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="prediction", metricName="weightedPrecision"
)
evaluator_recall = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="prediction", metricName="weightedRecall"
)

accuracy = evaluator_accuracy.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
metrics_time = time.time() - metrics_start
print(f"Metrics calculation time: {metrics_time:.2f} seconds")

evaluation_time = time.time() - evaluation_start

print(f"\n{'='*60}")
print(f"MODEL PERFORMANCE RESULTS")
print(f"{'='*60}")
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"F1 Score:  {f1_score:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"\nTotal evaluation time: {evaluation_time:.2f} seconds")


# Test with ensemble method

In [ ]:
# Ensemble approach with multiple classifiers
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

# Create multiple models
lr = LogisticRegression(
    featuresCol="pca_features",
    labelCol="label_index",
    maxIter=100,
    regParam=0.01
)

dt = DecisionTreeClassifier(
    featuresCol="pca_features",
    labelCol="label_index",
    maxDepth=10,
    minInstancesPerNode=1,
    seed=CONFIG["seed"]
)

# Train all models
print("Training ensemble models...")
lr_model = lr.fit(train_df)
dt_model = dt.fit(train_df)

# Make predictions with all models
rf_predictions = rf_model.transform(test_df)
lr_predictions = lr_model.transform(test_df)
dt_predictions = dt_model.transform(test_df)

# Ensemble voting (majority vote)
from pyspark.sql.functions import when, col

ensemble_predictions = rf_predictions.select("path", "label_index", 
                                           col("prediction").alias("rf_pred")) \
    .join(lr_predictions.select("path", col("prediction").alias("lr_pred")), "path") \
    .join(dt_predictions.select("path", col("prediction").alias("dt_pred")), "path")

# Majority voting
ensemble_predictions = ensemble_predictions.withColumn(
    "prediction",
    when((col("rf_pred") == col("lr_pred")) | (col("rf_pred") == col("dt_pred")), col("rf_pred"))
    .when(col("lr_pred") == col("dt_pred"), col("lr_pred"))
    .otherwise(col("rf_pred"))  # Default to RF if no majority
)

# Evaluate ensemble
ensemble_accuracy = evaluator_accuracy.evaluate(ensemble_predictions)
print(f"\nEnsemble Model Accuracy: {ensemble_accuracy:.4f} ({ensemble_accuracy*100:.2f}%)")


# Save Results and Summary

In [ ]:
# Calculate total time
total_time = time.time() - start_time

# Helper function to convert numpy types to Python native types
def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    else:
        return obj

# Prepare results with detailed timing breakdown
results = {
    "platform": {
        "system": PLATFORM,
        "is_emr": IS_EMR,
        "spark_version": spark.version,
        "python_version": sys.version.split()[0],
        "executors": int(sc._jsc.sc().getExecutorMemoryStatus().size()),
        "default_parallelism": int(sc.defaultParallelism)
    },
    "data": {
        "total_images": int(image_count),
        "processed_images": int(processed_count),
        "train_samples": int(train_count),
        "test_samples": int(test_count),
        "num_classes": len(label_indexer_model.labels),
        "classes": label_indexer_model.labels
    },
    "feature_extraction": {
        "method": "MobileNetV2 Transfer Learning",
        "input_size": list(CONFIG["image_size"]),
        "feature_dimension": int(CONFIG["num_features"]),
        "weights_broadcast_size_mb": float(len(serialized_weights) / (1024*1024))
    },
    "pca_analysis": {
        "components_used": int(CONFIG["pca_components"]),
        "variance_explained": float(cumulative_variance[-1]),
        "components_for_threshold": int(n_components_threshold)
    },
    "model": {
        "type": "RandomForestClassifier",
        "num_trees": int(CONFIG["num_trees"]),
        "max_depth": int(CONFIG["max_depth"]),
        "input_features": int(CONFIG["pca_components"])
    },
    "performance": {
        "accuracy": float(accuracy),
        "f1_score": float(f1_score),
        "precision": float(precision),
        "recall": float(recall),
        "ensemble_accuracy": float(ensemble_accuracy) if 'ensemble_accuracy' in locals() else None
    },
    "timing": {
        "initialization": {
            "spark_init": float(initialization_time),
            "model_load_broadcast": float(model_load_time),
            "total": float(initialization_time + model_load_time)
        },
        "data_loading": {
            "image_reading": float(image_read_time) if 'image_read_time' in locals() else None,
            "label_extraction": float(label_extraction_time) if 'label_extraction_time' in locals() else None,
            "validation": float(validation_time) if 'validation_time' in locals() else None,
            "caching": float(cache_time) if 'cache_time' in locals() else None,
            "counting": float(count_time) if 'count_time' in locals() else None,
            "total": float(loading_time)
        },
        "feature_extraction": {
            "total": float(feature_extraction_time)
        },
        "pca_analysis": {
            "total": float(pca_time)
        },
        "model_training": {
            "preparation": float(preparation_time) if 'preparation_time' in locals() else None,
            "training": float(training_time),
            "total": float(training_time + (preparation_time if 'preparation_time' in locals() else 0))
        },
        "evaluation": {
            "prediction": float(prediction_time) if 'prediction_time' in locals() else None,
            "metrics": float(metrics_time) if 'metrics_time' in locals() else None,
            "total": float(evaluation_time)
        },
        "total_execution_time": float(total_time)
    },
    "timestamp": datetime.now().isoformat()
}

# Convert all values to be JSON serializable
results = convert_to_serializable(results)

# Print detailed timing summary
print(f"\n{'='*80}")
print(f"DETAILED TIMING BREAKDOWN")
print(f"{'='*80}")
print(f"\n1. Initialization Phase:")
print(f"   - Spark initialization: {results['timing']['initialization']['spark_init']:.2f}s")
print(f"   - Model load & broadcast: {results['timing']['initialization']['model_load_broadcast']:.2f}s")
print(f"   - Subtotal: {results['timing']['initialization']['total']:.2f}s")

print(f"\n2. Data Loading Phase:")
if results['timing']['data_loading']['image_reading']:
    print(f"   - Image reading: {results['timing']['data_loading']['image_reading']:.2f}s")
    print(f"   - Label extraction: {results['timing']['data_loading']['label_extraction']:.2f}s")
    print(f"   - Validation: {results['timing']['data_loading']['validation']:.2f}s")
    print(f"   - Caching: {results['timing']['data_loading']['caching']:.2f}s")
    print(f"   - Counting: {results['timing']['data_loading']['counting']:.2f}s")
print(f"   - Subtotal: {results['timing']['data_loading']['total']:.2f}s")

print(f"\n3. Feature Extraction Phase:")
print(f"   - Total: {results['timing']['feature_extraction']['total']:.2f}s")
print(f"   - Per image: {results['timing']['feature_extraction']['total']/results['data']['processed_images']*1000:.2f}ms")

print(f"\n4. PCA Analysis Phase:")
print(f"   - Total: {results['timing']['pca_analysis']['total']:.2f}s")

print(f"\n5. Model Training Phase:")
if results['timing']['model_training']['preparation']:
    print(f"   - Preparation: {results['timing']['model_training']['preparation']:.2f}s")
print(f"   - Training: {results['timing']['model_training']['training']:.2f}s")
print(f"   - Subtotal: {results['timing']['model_training']['total']:.2f}s")

print(f"\n6. Evaluation Phase:")
if results['timing']['evaluation']['prediction']:
    print(f"   - Prediction: {results['timing']['evaluation']['prediction']:.2f}s")
    print(f"   - Metrics calculation: {results['timing']['evaluation']['metrics']:.2f}s")
print(f"   - Subtotal: {results['timing']['evaluation']['total']:.2f}s")

print(f"\n{'='*80}")
print(f"TOTAL EXECUTION TIME: {results['timing']['total_execution_time']:.2f}s ({results['timing']['total_execution_time']/60:.2f} minutes)")
print(f"{'='*80}")

# Performance summary
print(f"\n{'='*80}")
print(f"PERFORMANCE SUMMARY")
print(f"{'='*80}")
print(f"Platform: {results['platform']['system']} {'(EMR)' if IS_EMR else '(Local)'}")
print(f"Feature Extraction: {results['feature_extraction']['method']}")
print(f"Model Broadcast Size: {results['feature_extraction']['weights_broadcast_size_mb']:.2f} MB")
print(f"\nPCA Analysis:")
print(f"  - Components used: {results['pca_analysis']['components_used']}")
print(f"  - Variance explained: {results['pca_analysis']['variance_explained']:.2%}")
print(f"\nModel Performance:")
print(f"  - Accuracy:  {results['performance']['accuracy']:.2%}")
print(f"  - F1 Score:  {results['performance']['f1_score']:.4f}")
print(f"  - Precision: {results['performance']['precision']:.4f}")
print(f"  - Recall:    {results['performance']['recall']:.4f}")
if results['performance']['ensemble_accuracy']:
    print(f"  - Ensemble Accuracy: {results['performance']['ensemble_accuracy']:.2%}")
print(f"{'='*80}")

# Save results with error handling
try:
    if not IS_EMR:
        os.makedirs(RESULTS_PATH, exist_ok=True)
        results_file = os.path.join(RESULTS_PATH, f"results_transfer_learning_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        
        # Alternative method using custom JSON encoder
        class NumpyEncoder(json.JSONEncoder):
            def default(self, obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                elif isinstance(obj, np.floating):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                return super(NumpyEncoder, self).default(obj)
        
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=2, cls=NumpyEncoder)
        
        print(f"\nResults saved to: {results_file}")
    else:
        # For EMR, you might want to save to S3
        print("\nRunning on EMR - implement S3 save if needed")
        
except Exception as e:
    print(f"\nWarning: Could not save results file: {str(e)}")
    print("Results printed to console instead.")


# Cleanup

In [ ]:
# Clean up broadcasts
broadcast_weights.unpersist()
broadcast_config.unpersist()

# Stop Spark session
spark.stop()
print("Spark session closed.")
print("\nExecution completed successfully!")
